# Testing bounding

In [1]:
import sys; sys.path.append('..')
import cadence as cd
from cadence.imports import *

In [3]:
line="Of that Forbidden Tree, whose mortal tast"

In [4]:
# cd.scan(line,phrasebreaks=False)

In [6]:
dfparses=cd.parse(line,phrasebreaks=False)
dfparses

Iterating over line scansions [x1]: 100%|██████████| 1/1 [00:00<00:00, 242.80it/s]


,,,,,,,,,,,*total,*clash,*lapse,*w/stressed,*s/unstressed,*w/peak,*w-res,*f-res,combo_num_syll,is_funcword,is_heavy,is_light,is_peak,is_s,is_stressed,is_syll,is_trough,is_unstressed,is_w,line_num_syll,parse_num_pos,parse_num_syll,parse_rank_dense,parse_rank_min,prom_strength,prom_stress,prom_weight
stanza_i,line_i,linepart_i,line_str,parse_rank,combo_i,combo_stress,combo_ipa,parse_i,parse,parse_str,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,1,"Of that Forbidden Tree, whose mortal tast",1,3,UPUPUPUPUP,ʌv 'ðæt fɛːr.'bɪ.dən 'triː huːz 'mɔːr.təl 'teɪst,1,wswswswsws,of|THAT|for|BID|den|TREE|whose|MOR|tal|TAST,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,2,8,2,2,5,5,10,3,5,5,10,10,10,1,1,2.0,5.0,8


In [5]:
for linedf in cd.iter_lines(line,phrasebreaks=False): pass
# linedf

Iterating over line scansions [x1]: 100%|██████████| 1/1 [00:00<00:00, 224.04it/s]


In [6]:
linedf_parsed=cd.do_iter_parsed_lines(linedf)

In [38]:

def postproc_dfline(dfline,by_line=False,only_best=False,only_unbound=True):
    odf=to_lines(dfline) if by_line else sort_by_total_and_syll(dfline)
    if only_unbound: odf=bound_parses(odf)
    if only_best and 'parse_rank' in set(odf.index.names) or 'parse_rank' in set(odf.columns):
        odf=odf.query('parse_rank==1')
    return odf

def is_bounded(pdf1,pdf2,totalcol=TOTALCOL,cprefix='*'):
    cdf1=pdf1[[c for c in pdf1.columns if c.startswith(cprefix) and c!=totalcol]]
    cdf2=pdf2[[c for c in pdf2.columns if c.startswith(cprefix) and c!=totalcol]]
    c1=cdf1.sum()
    c2=cdf2.sum()
    c1=c1[c1>0]
    c2=c2[c2>0]
    set1,set2=set(c1.index),set(c2.index)
    return set1.issubset(set2), set2.issubset(set1)


def bound_parses(odf):
    grps=sorted(list(odf.groupby('parse_rank')))
    not_ok_ranks=set()
    for parse_rank1,parsedf1 in grps:
        for parse_rank2,parsedf2 in grps:
            if parse_rank1>=parse_rank2: continue
            x_bounds_y, y_bounds_x = is_bounded(parsedf1,parsedf2)
            if y_bounds_x: not_ok_ranks|={parse_rank1}
            if x_bounds_y: not_ok_ranks|={parse_rank2}
    return odf[~odf.index.get_level_values('parse_rank').isin(not_ok_ranks)]

In [41]:
odf=postproc_dfline(linedf_parsed)
odf

*total  ...  prom_weight
stanza_i line_i linepart_i line_str                                  parse_rank combo_i combo_stress combo_ipa                                        parse_i parse      parse_str                                   parse_pos_i parse_pos word_i word_str  word_ipa_i word_ipa     syll_i combo_syll_i syll_str syll_ipa syll_stress syll_weight parse_syll_i parse_syll          ...             
1        1      1          Of that Forbidden Tree, whose mortal tast 1          3       UPUPUPUPUP   ʌv 'ðæt fɛːr.'bɪ.dən 'triː huːz 'mɔːr.təl 'teɪst 1       wswswswsws of|THAT|for|BID|den|TREE|whose|MOR|tal|TAST 0           w         1      Of        1          ʌv           1      0            Of       ʌv       U           H           1            w              0.0  ...            1
                                                                                                                                                                                                                     1           s         2      that      1          'ðæt         1      1            that     'ðæt     P           H           1            s              0.0  ...            1
                                                                                                                                                                                                                     2           w         3      Forbidden 2          fɛːr.'bɪ.dən 1      2            For      fɛːr     U           H           1            w              0.0  ...            1
                                                                                                                                                                                                                     3           s         3      Forbidden 2          fɛːr.'bɪ.dən 2      3            bid      'bɪ      P           L           1            s              0.0  ...            0
                                                                                                                                                                                                                     4           w         3      Forbidden 2          fɛːr.'bɪ.dən 3      4            den      dən      U           H           1            w              0.0  ...            1
                                                                                                                                                                                                                     5           s         4      Tree      1          'triː        1      5            Tree     'triː    P           L           1            s              0.0  ...            0
                                                                                                                                                                                                                     6           w         5      whose     2          huːz         1      6            whose    huːz     U           H           1            w              0.0  ...            1
                                                                                                                                                                                                                     7           s         6      mortal    1          'mɔːr.təl    1      7            mor      'mɔːr    P           H           1            s              0.0  ...            1
                                                                                                                                                                                                                     8           w         6      mortal    1          'mɔːr.təl    2      8            tal      təl      U           H           1            w              0.0  ...            1
                                                                                               